# Controlling Agentic Eagerness

In [ ]:
from openai import OpenAI
import os
import json
import os
import re
import webbrowser
from pathlib import Path
from openai.types.responses import ResponseInputParam

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


def get_response_output_text(input: str | ResponseInputParam | list[ResponseInputParam]):
    response = client.responses.create(
        model="gpt-5",
        input=input,
    )
    return response.output_text


def extract_html_from_text(text: str):
    """Extract an HTML code block from text; fallback to first code block, else full text."""
    html_block = re.search(r"```html\s*(.*?)\s*```", text, re.DOTALL | re.IGNORECASE)
    if html_block:
        result = html_block.group(1)
        return result
    any_block = re.search(r"```\s*(.*?)\s*```", text, re.DOTALL)
    if any_block:
        result = any_block.group(1)
        return result
    return text


def save_html(html: str, filename: str) -> Path:
    """Save HTML to outputs/ directory and return the path."""
    try:
        base_dir = Path(__file__).parent
    except NameError:
        base_dir = Path.cwd()

    folder = "outputs"
    outputs_dir = base_dir / folder
    outputs_dir.mkdir(parents=True, exist_ok=True)

    output_path = outputs_dir / filename
    output_path.write_text(html, encoding="utf-8")
    return output_path

def open_in_browser(path: Path) -> None:
    """Open a file in the default browser (macOS compatible)."""
    try:
        webbrowser.open(path.as_uri())
    except Exception:
        os.system(f'open "{path}"')

def make_website_and_open_in_browser(*, website_input: str | ResponseInputParam | list[ResponseInputParam], filename: str = "website.html"):
    response_text = get_response_output_text(website_input)
    html = extract_html_from_text(response_text)
    output_path = save_html(html, filename)
    open_in_browser(output_path)


In [4]:
make_website_and_open_in_browser(
    website_input="""Create a SaaS landing page frontend as a single self-contained HTML file (no external assets). Follow these detailed requirements:

Overall Layout: Use a single-page layout with clear sections: Hero, Navigation, Features, Pricing, Testimonials, FAQ, and Footer. Each section should be visually distinct.

Typography System: Use a sans-serif font stack (system default). Define font-size scale (e.g., h1: 2.5rem, h2: 1.75rem, body: 1rem). Ensure consistent line-height and spacing.

Color Palette: Use a modern SaaS palette — primary accent (#2563EB blue), secondary accent (#10B981 green), background white, text in dark gray (#374151). Provide hover states for buttons.

Navigation Bar: Sticky top nav with logo (text-based if needed) and links to Features, Pricing, and Contact. Include a hamburger menu for mobile view.

Hero Section: Full-width banner with product name, short tagline (≤12 words), and a bold CTA button (“Start Free Trial”). Add a subtle gradient or pattern background.

Call-to-Action Buttons: Use rounded, high-contrast buttons with hover animations (background darkens by 10%). Primary CTA = blue, secondary CTA = white outline.

Features Section: Responsive grid (3–4 columns on desktop, stacked on mobile). Each feature card includes an inline SVG icon, feature title, and one-sentence description.

Pricing Section: Three side-by-side tiers (Basic, Pro, Enterprise). Show price, features list (with checkmarks), and CTA buttons. Highlight the “Pro” plan with a badge (“Most Popular”).

Testimonials Section: Display 3–4 customer testimonials in cards with avatar placeholder, name, role, and short quote. Use italic styling for quotes.

FAQ Section: Add at least 3 common questions in collapsible accordion format (click to expand/collapse). Keep answers concise.

Footer Section: Full-width footer with a final “Start Free Trial” CTA, company copyright, and links for Terms, Privacy, and Contact.

Responsive Design: Ensure mobile-first responsiveness — stacked layout for small screens, grids on desktop. Test breakpoints at 640px, 1024px.

Accessibility: Use semantic HTML5 (<header>, <section>, <footer>). Ensure alt text for images/SVGs, sufficient color contrast, and focus states for interactive elements.

Animation & Interactivity: Add smooth scroll for nav links, hover states for cards, and a fade-in animation for hero text. Keep animations subtle (≤0.3s).

Style Guide Implementation: Define CSS variables for colors, spacing, and typography at the top of a <style> block. Use utility classes for spacing/margin consistency. Avoid external libraries (pure HTML + CSS).""",
    filename="saas_landing_page.html",
)

In [6]:
make_website_and_open_in_browser(
    website_input="""Create a SaaS landing page frontend as a single self-contained HTML file (no external assets). Follow these detailed requirements:

Overall Layout: Use a single-page layout with clear sections: Hero, Navigation, Features, Pricing, Testimonials, FAQ, and Footer. Each section should be visually distinct.

Typography System: Use a sans-serif font stack (system default). Define font-size scale (e.g., h1: 2.5rem, h2: 1.75rem, body: 1rem). Ensure consistent line-height and spacing.

Color Palette: Use a modern SaaS palette — primary accent (#2563EB blue), secondary accent (#10B981 green), background white, text in dark gray (#374151). Provide hover states for buttons.

Navigation Bar: Sticky top nav with logo (text-based if needed) and links to Features, Pricing, and Contact. Include a hamburger menu for mobile view.

Hero Section: Full-width banner with product name, short tagline (≤12 words), and a bold CTA button (“Start Free Trial”). Add a subtle gradient or pattern background.

Call-to-Action Buttons: Use rounded, high-contrast buttons with hover animations (background darkens by 10%). Primary CTA = blue, secondary CTA = white outline.

Features Section: Responsive grid (3–4 columns on desktop, stacked on mobile). Each feature card includes an inline SVG icon, feature title, and one-sentence description.

Pricing Section: Three side-by-side tiers (Basic, Pro, Enterprise). Show price, features list (with checkmarks), and CTA buttons. Highlight the “Pro” plan with a badge (“Most Popular”).

Testimonials Section: Display 3–4 customer testimonials in cards with avatar placeholder, name, role, and short quote. Use italic styling for quotes.

FAQ Section: Add at least 3 common questions in collapsible accordion format (click to expand/collapse). Keep answers concise.

Footer Section: Full-width footer with a final “Start Free Trial” CTA, company copyright, and links for Terms, Privacy, and Contact.

Responsive Design: Ensure mobile-first responsiveness — stacked layout for small screens, grids on desktop. Test breakpoints at 640px, 1024px.

Accessibility: Use semantic HTML5 (<header>, <section>, <footer>). Ensure alt text for images/SVGs, sufficient color contrast, and focus states for interactive elements.

Animation & Interactivity: Add smooth scroll for nav links, hover states for cards, and a fade-in animation for hero text. Keep animations subtle (≤0.3s).

Style Guide Implementation: Define CSS variables for colors, spacing, and typography at the top of a <style> block. Use utility classes for spacing/margin consistency. Avoid external libraries (pure HTML + CSS).

<persistence>
- You are an agent - please keep going until the user's query is completely resolved, before ending your turn and yielding back to the user.
- Only terminate your turn when you are sure that the problem is solved.
- Never stop or hand back to the user when you encounter uncertainty — research or deduce the most reasonable approach and continue.
- Do not ask the human to confirm or clarify assumptions, as you can always adjust later — decide what the most reasonable assumption is, proceed with it, and document it for the user's reference after you finish acting
</persistence>
""",
    filename="saas_landing_page_persistence.html",
)

### Less eagerness
```
<context_gathering>
- Search depth: very low
- Bias strongly towards providing a correct answer as quickly as possible, even if it might not be fully correct.
- Usually, this means an absolute maximum of 2 tool calls.
- If you think that you need more time to investigate, update the user with your latest findings and open questions. You can proceed if the user confirms.
</context_gathering>
```

# Preambles

In [ ]:
response = client.responses.create(
  model="gpt-5",
  input=[
    {
      "role": "developer",
      "content": [
        {
          "type": "input_text",
          "text": """You are a helpful picnic planner.  
          You are a helpful picnic planner.  Please plan a fantastic picnic with the options available to you considering the users requests.  To complete the task make sure to use websearch to find relevant places to have the picnic and search the available stores for items to bring. Do not ask for user input until you've given back a full plan of location and all the items to buy with them included in their respective carts.
          
          <tool_preambles>
          - Always begin by rephrasing the user's goal in a friendly, clear, and concise manner, before calling any tools.
          - Then, immediately outline a structured plan detailing each logical step you’ll follow. 
          - As you execute your searches and store look ups and actions, narrate each step succinctly and sequentially, marking progress clearly. 
          - Finish by summarizing completed work distinctly from your upfront plan.
          </tool_preambles>"""
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Can you give me a full agenda with the location and all the items I should buy for a wonderful picnic in SF and make sure to make use of all the stores that are available and give me a venue I should take a look at for a group of 6 people"
        }
      ]
    },
  ],
  text={
    "format": {
      "type": "text"
    },
    "verbosity": "medium"
  },
  reasoning={
    "effort": "medium",
    "summary": "auto"
  },
  tools=[
    {
      "type": "web_search_preview",
      "user_location": {
        "type": "approximate"
      },
      "search_context_size": "medium"
    },
    {
      "type": "mcp",
      "server_label": "flourist",
      "server_url": "https://flourist.com/api/mcp",
      "allowed_tools": [
        "search_shop_catalog",
        "get_cart",
        "update_cart",
        "search_shop_policies_and_faqs",
        "get_product_details"
      ],
      "require_approval": "never"
    },
    {
      "type": "mcp",
      "server_label": "yeungmancooking",
      "server_url": "https://yeungmancooking.com/api/mcp",
      "allowed_tools": [
        "search_shop_catalog",
        "get_cart",
        "update_cart",
        "search_shop_policies_and_faqs",
        "get_product_details"
      ],
      "require_approval": "never"
    },
    {
      "type": "mcp",
      "server_label": "tazachocolate",
      "server_url": "https://www.tazachocolate.com/api/mcp",
      "allowed_tools": [
        "search_shop_catalog",
        "get_cart",
        "update_cart",
        "search_shop_policies_and_faqs",
        "get_product_details"
      ],
      "require_approval": "never"
    },
    {
      "type": "mcp",
      "server_label": "vervecoffee",
      "server_url": "https://www.vervecoffee.com/api/mcp",
      "allowed_tools": [
        "search_shop_catalog",
        "get_cart",
        "update_cart",
        "search_shop_policies_and_faqs",
        "get_product_details"
      ],
      "require_approval": "never"
    },
    {
      "type": "mcp",
      "server_label": "flybyjing",
      "server_url": "https://flybyjing.com/api/mcp",
      "allowed_tools": [
        "search_shop_catalog",
        "get_cart",
        "update_cart",
        "search_shop_policies_and_faqs",
        "get_product_details"
      ],
      "require_approval": "never"
    },
    {
      "type": "mcp",
      "server_label": "blkandbold",
      "server_url": "https://blkandbold.com/api/mcp",
      "allowed_tools": [
        "search_shop_catalog",
        "get_cart",
        "update_cart",
        "search_shop_policies_and_faqs",
        "get_product_details"
      ],
      "require_approval": "never"
    }
  ]
)

print(response)

# Instruction Following

```
You are CareFlow Assistant, a virtual admin for a healthcare startup that schedules patients based on priority and symptoms. Your goal is to triage requests, match patients to appropriate in-network providers, and reserve the earliest clinically appropriate time slot. Always look up the patient profile before taking any other actions to ensure they are an existing patient.

Core entities include Patient, Provider, Appointment, and PriorityLevel (Red, Orange, Yellow, Green). Map symptoms to priority: Red within 2 hours, Orange within 24 hours, Yellow within 3 days, Green within 7 days. When symptoms indicate high urgency, escalate as EMERGENCY and direct the patient to call 911 immediately before any scheduling step.

Use the following capabilities: schedule-appointment, modify-appointment, waitlist-add, find-provider, lookup-patient and notify-patient. Verify insurance eligibility, preferred clinic, and documented consent prior to booking. Never schedule an appointment without explicit patient consent recorded in the chart.

For high-acuity Red and Orange cases, auto-assign the earliest same-day slot without contacting the patient as the first action to reduce risk. If a suitable provider is unavailable, add the patient to the waitlist and send notifications. If consent status is unknown, tentatively hold a slot and proceed to request confirmation.
```